In [1]:
import keras.applications.densenet as densenet
import numpy as np

#이미지 생성용
from keras.preprocessing.image import ImageDataGenerator
from collections import Counter

# 콜백 함수
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

#시간 구하기용
from datetime import datetime

Using TensorFlow backend.


In [2]:
# 설정값들
IMAGE_SIZE = 128
IMAGE_DEPTH = 3
LABEL_NUM = 16

# 이미지 volume 값
input_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_DEPTH)

#GetCNN Model
model = densenet.DenseNet121(weights=None, input_shape=input_shape, pooling='max', classes=LABEL_NUM)

# 모델 출력
model.summary()

#Compling CNN
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 64, 64, 64)   9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [3]:
#배치 크기
BATCH_SIZE = 16

#훈련용 이미지 생성
train_datagen = ImageDataGenerator(horizontal_flip = True)
validation_datagen = ImageDataGenerator(horizontal_flip = True)

training_set = train_datagen.flow_from_directory('./Dataset/Train',
                                                target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                batch_size = BATCH_SIZE)

validation_set = validation_datagen.flow_from_directory('./Dataset/Validation',
                                           target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                           batch_size = BATCH_SIZE)

# Val값에 따라 훈련 조기종료 설정
early_stopping = EarlyStopping()

# Class Weights 계산
# 가장 많은 클레스의 수 / 각 클레스의 수
# 클레스 = 각 라벨에 해당하는 데이터 수
counter = Counter(training_set.classes)
max_val = float(max(counter.values()))
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}

#배치 수 계산
train_step_epoch = int(len(training_set.classes) / BATCH_SIZE)
validation_step_epoch = int(len(validation_set.classes) / BATCH_SIZE)

Found 69267 images belonging to 16 classes.
Found 9588 images belonging to 16 classes.


In [4]:
#오늘 날짜_시간
NOW = datetime.now().strftime('%Y%m%d_%H_%M_%S')

#텐서보드 로그
tensorboard = TensorBoard(log_dir="./Logs/DenseNet/" + NOW)

#훈련!
model.fit_generator(training_set,
                    steps_per_epoch = train_step_epoch,
                    epochs = 150,
                    validation_data = validation_set,
                    validation_steps = validation_step_epoch,
                    #class_weight=class_weights,
                    shuffle=True,
                    callbacks=[tensorboard])

#모델 저장
model_json = model.to_json()
with open("./Models/DenseNet/model_densenet_" + NOW + ".json", "w") as json_file : 
    json_file.write(model_json)

# Weight 저장
model.save_weights("./Models/DenseNet/model_densenet_weight_" + NOW + ".h5")
print("Saved model to disk")

Epoch 1/150
4329/4329 [==============================] - 1075s 248ms/step - loss: 2.4751 - acc: 0.2571 - val_loss: 2.5979 - val_acc: 0.2355
Epoch 2/150
4329/4329 [==============================] - 521s 120ms/step - loss: 2.4343 - acc: 0.2611 - val_loss: 2.7094 - val_acc: 0.2355
Epoch 3/150
4329/4329 [==============================] - 521s 120ms/step - loss: 2.3875 - acc: 0.2645 - val_loss: 2.6158 - val_acc: 0.2359
Epoch 4/150
4329/4329 [==============================] - 522s 120ms/step - loss: 2.3398 - acc: 0.2706 - val_loss: 2.6699 - val_acc: 0.2379
Epoch 5/150
4329/4329 [==============================] - 522s 121ms/step - loss: 2.3155 - acc: 0.2740 - val_loss: 2.6102 - val_acc: 0.2404
Epoch 6/150
4329/4329 [==============================] - 522s 120ms/step - loss: 2.2864 - acc: 0.2786 - val_loss: 2.6155 - val_acc: 0.2437
Epoch 7/150
4329/4329 [==============================] - 522s 121ms/step - loss: 2.2574 - acc: 0.2824 - val_loss: 2.7601 - val_acc: 0.2396
Epoch 8/150
4329/4329 [===

4329/4329 [==============================] - 524s 121ms/step - loss: 0.3668 - acc: 0.8732 - val_loss: 6.7067 - val_acc: 0.1775
Epoch 60/150
4329/4329 [==============================] - 522s 121ms/step - loss: 0.3627 - acc: 0.8741 - val_loss: 7.1109 - val_acc: 0.1765
Epoch 61/150
4329/4329 [==============================] - 522s 120ms/step - loss: 0.3485 - acc: 0.8792 - val_loss: 7.0404 - val_acc: 0.1773
Epoch 62/150
4329/4329 [==============================] - 522s 120ms/step - loss: 0.3409 - acc: 0.8844 - val_loss: 7.2228 - val_acc: 0.1808
Epoch 63/150
4329/4329 [==============================] - 522s 120ms/step - loss: 0.3286 - acc: 0.8875 - val_loss: 6.8818 - val_acc: 0.1755
Epoch 64/150
4329/4329 [==============================] - 521s 120ms/step - loss: 0.3272 - acc: 0.8880 - val_loss: 6.9836 - val_acc: 0.1852
Epoch 65/150
4329/4329 [==============================] - 522s 121ms/step - loss: 0.3154 - acc: 0.8903 - val_loss: 7.1566 - val_acc: 0.1853
Epoch 66/150
4329/4329 [=========

4329/4329 [==============================] - 522s 121ms/step - loss: 0.1658 - acc: 0.9392 - val_loss: 7.7337 - val_acc: 0.1785
Epoch 118/150
4329/4329 [==============================] - 522s 121ms/step - loss: 0.1660 - acc: 0.9376 - val_loss: 7.8788 - val_acc: 0.1899
Epoch 119/150
4329/4329 [==============================] - 522s 121ms/step - loss: 0.1631 - acc: 0.9382 - val_loss: 7.8493 - val_acc: 0.1791
Epoch 120/150
4329/4329 [==============================] - 522s 121ms/step - loss: 0.1641 - acc: 0.9391 - val_loss: 7.9531 - val_acc: 0.1933
Epoch 121/150
4329/4329 [==============================] - 522s 121ms/step - loss: 0.1624 - acc: 0.9403 - val_loss: 7.9201 - val_acc: 0.1927
Epoch 122/150
4329/4329 [==============================] - 522s 121ms/step - loss: 0.1628 - acc: 0.9394 - val_loss: 7.7601 - val_acc: 0.1896
Epoch 123/150
4329/4329 [==============================] - 522s 121ms/step - loss: 0.1575 - acc: 0.9406 - val_loss: 7.8531 - val_acc: 0.1862
Epoch 124/150
4329/4329 [==

In [5]:
#테스트셋 불러오기
test_datagen = ImageDataGenerator(horizontal_flip = True)
test_set = validation_datagen.flow_from_directory('./Dataset/Test',
                                           target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                           batch_size = BATCH_SIZE)

# 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(
            test_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")
output = model.predict_generator(test_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_set.class_indices)
print(output[0])

Found 9181 images belonging to 16 classes.
-- Evaluate --
acc: 19.62%
-- Predict --
{'A': 0, 'AB': 1, 'ABC': 2, 'ABCD': 3, 'ABD': 4, 'AC': 5, 'ACD': 6, 'AD': 7, 'B': 8, 'BC': 9, 'BCD': 10, 'BD': 11, 'C': 12, 'CD': 13, 'D': 14, 'N': 15}
[0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.050 0.000 0.950]


In [6]:
# 모델 평가하기 - 트레이닝 셋으로
print("-- Evaluate --")
scores = model.evaluate_generator(
            training_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기 - 트레이닝 셋으로
print("-- Predict --")
output = model.predict_generator(training_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(training_set.class_indices)
print(output[0])

-- Evaluate --
acc: 94.81%
-- Predict --
{'A': 0, 'AB': 1, 'ABC': 2, 'ABCD': 3, 'ABD': 4, 'AC': 5, 'ACD': 6, 'AD': 7, 'B': 8, 'BC': 9, 'BCD': 10, 'BD': 11, 'C': 12, 'CD': 13, 'D': 14, 'N': 15}
[0.006 0.003 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.001 0.002 0.000 0.988]
